# NONMEM Dataset Builder (Portfolio-safe)

This notebook is a sanitized version of my internal workflow for building NONMEM-ready datasets.
- Uses **relative paths** (`INPUT_DIR`, `OUTPUT_DIR`) instead of `os.chdir`.
- Expects **public/synthetic** Excel files in `data/public/` (e.g., `tbl500.xlsx`, `tblPt.xlsx`, `tblTNF.xlsx`).
- Writes outputs to `derived/nonmem_notebook/`.
- No confidential data or absolute paths included.


In [ ]:
import os
import pandas as pd
import numpy as np

INPUT_DIR = 'data/public'           # place demo Excel files here
OUTPUT_DIR = 'derived/nonmem_notebook'
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Anchor date used in original analysis (kept for reproducibility)
ANCHOR_DATE = pd.Timestamp('2000-01-01')
print('Writing outputs to:', OUTPUT_DIR)


In [ ]:
import pandas as pd
import os
import numpy as np

print(os.getcwd())

tbl500 = pd.pd.read_excel(os.path.join(INPUT_DIR, r'tbl500.xlsx'))
tblpt2 = pd.pd.read_excel(os.path.join(INPUT_DIR, r'tblPt2.xlsx'))
tblTNF = pd.pd.read_excel(os.path.join(INPUT_DIR, r'tblTNF.xlsx'))
tblpt = pd.pd.read_excel(os.path.join(INPUT_DIR, r'tblPt.xlsx'))


## With extra rows

In [ ]:

tbl500 = tbl500[tbl500["Days"] > 0]


# Ensure 'MapNummerOld' exists before renaming
if 'MapNummerOld' not in tbl500.columns:
    raise KeyError("Column 'MapNummerOld' not found in tbl500.xlsx")
if 'mapnummer' not in tblpt.columns:
    raise KeyError("Column 'mapnummer' not found in tblPt.xlsx")

# Convert date columns to datetime
start_date = pd.Timestamp("2000-01-01")
tbl500["Afnamedatum"] = pd.to_datetime(tbl500["Afnamedatum"], errors='coerce')
tbl500["Eerste prik"] = pd.to_datetime(tbl500["Eerste prik"], errors='coerce')

# Calculate TIME (days since 01-01-2000) and TSFD (days since Eerste prik)
tbl500["TIME"] = (tbl500["Afnamedatum"] - start_date).dt.days
tbl500["TSFD"] = (tbl500["Afnamedatum"] - tbl500["Eerste prik"]).dt.days

# Extract VISIT (numeric value only)
tbl500["VISIT"] = tbl500["Visit"].str.extract(r'(\d+)').astype(float)

# Assign fixed values
TAD_measurement = 14
TAD_dose = "."
CMT_measurement = 2
CMT_dose = 1
EVID_measurement = 0
EVID_dose = 4
MDV = 0
SS = "."
II = 14  # Dosing every 14 days
AMT_measurement = "."
AMT_dose = 40

# Rename columns for consistency
tbl500 = tbl500.rename(columns={
    "MapNummerOld": "NMID", 
    "adalimumab": "DV", 
    "ARIAtxt": "ADA"
})
tblpt = tblpt.rename(columns={"mapnummer": "NMID", "weight baseline": "WT"})

# Replace missing ADA values with '.'
tbl500["ADA"] = tbl500["ADA"].astype(str).str.extract(r'(\d+)')[0].fillna(".")

# Merge MTXdosis and Weight from tblPt
merged_data = tbl500.merge(tblpt[["NMID", "MTXdosis", "WT"]], on="NMID", how="left")

# Fill missing MTXdosis with 0
merged_data["MTXdosis"] = merged_data["MTXdosis"].fillna(0)

# Create measurement records (CMT = 2)
measurements = merged_data.copy()
measurements["CMT"] = CMT_measurement
measurements["MDV"] = 0
measurements["EVID"] = EVID_measurement
measurements["AMT"] = AMT_measurement
measurements["TAD"] = TAD_measurement

# Generate dosing records (CMT = 1)
dose_records = []
for nmid, group in merged_data.groupby("NMID"):
    eerste_prik = group["Eerste prik"].iloc[0]
    last_measurement_time = group["TIME"].max()
    dose_time = (eerste_prik - start_date).days
    dose_count = 0
    
    while dose_time <= last_measurement_time:
        dose_count += 1
        SS_value = 1 if dose_count >= 3 else "."
         # Determine SS based on VISIT condition
        SS_value = 1 if row["VISIT"] >= 16 else 0  # 1 for VISIT >= 28, else 0
        II_value = II_dose if SS_value == 1 else "."  # Set II to "." if SS = 0
        EVID_value = EVID_dose if SS_value == 1 else 1
        
        dose_records.append({
            "NMID": nmid,
            "TIME": dose_time,
            "TAD": TAD_dose,
            "DV": ".",
            "MDV": 1,
            "CMT": CMT_dose,
            "EVID": EVID_value,
            "AMT": AMT_dose,
            "SS": SS_value,
            "II": II_value,
            "VISIT": ".",
            "TSFD": dose_time - (eerste_prik - start_date).days,
            "WT": group["WT"].iloc[0],
            "ADA": ".",
            "MTXdosis": group["MTXdosis"].iloc[0]
        })
        dose_time += 14  # Next dose after 14 days

# Convert dosing records to DataFrame
doses = pd.DataFrame(dose_records)

# Concatenate dose and measurement records
final_dataset = pd.concat([measurements, doses], ignore_index=True)

# Ensure correct column order
column_order = ["NMID", "TIME", "TAD", "DV", "MDV", "CMT", "EVID", "AMT", "SS", "II", "VISIT", "TSFD", "WT", "ADA", "MTXdosis"]
final_dataset = final_dataset[column_order]

# Sort by NMID and TIME
final_dataset = final_dataset.sort_values(by=["NMID", "TIME"]).reset_index(drop=True)
# Replace all empty or missing values in the dataset with "."
final_dataset = final_dataset.map(lambda x: "." if pd.isna(x) or x == "" else x)


# Save as CSV
final_dataset.to_csv(os.path.join(OUTPUT_DIR, r'NONMEM_dataset2.csv'), index=False)


## Drop extra rw

In [ ]:
import pandas as pd
import numpy as np

# Load data
tbl500 = pd.pd.read_excel(os.path.join(INPUT_DIR, r'tbl500.xlsx'))
tblpt = pd.pd.read_excel(os.path.join(INPUT_DIR, r'tblPt.xlsx'))

# Ensure 'MapNummerOld' exists before renaming
if 'MapNummerOld' not in tbl500.columns:
    raise KeyError("Column 'MapNummerOld' not found in tbl500.xlsx")
if 'mapnummer' not in tblpt.columns:
    raise KeyError("Column 'mapnummer' not found in tblPt.xlsx")

# Convert date columns to datetime
start_date = pd.Timestamp("2000-01-01")
tbl500["Afnamedatum"] = pd.to_datetime(tbl500["Afnamedatum"], errors='coerce')
tbl500["Eerste prik"] = pd.to_datetime(tbl500["Eerste prik"], errors='coerce')

# Calculate TIME (days since 01-01-2000) and TSFD (days since Eerste prik)
tbl500["TIME"] = (tbl500["Afnamedatum"] - start_date).dt.days
tbl500["TSFD"] = (tbl500["Afnamedatum"] - tbl500["Eerste prik"]).dt.days

# Extract VISIT (numeric value only)
tbl500["VISIT"] = tbl500["Visit"].str.extract(r'(\d+)').astype(float)

# Assign fixed values
TAD_measurement = 14
TAD_dose = "."
CMT_measurement = 2
CMT_dose = 1
EVID_measurement = 0
EVID_dose = 4
MDV_measurement = 0
MDV_dose = 1
SS_dose = 1
AMT_measurement = "."
AMT_dose = 40
II_dose = 14

# Rename columns for consistency
tbl500 = tbl500.rename(columns={
    "MapNummerOld": "NMID", 
    "adalimumab": "DV", 
    "ARIAtxt": "ADA"
})
tblpt = tblpt.rename(columns={"mapnummer": "NMID", "weight baseline": "WT"})

# Replace missing ADA values with '.' and extract numeric values
tbl500["ADA"] = tbl500["ADA"].astype(str).str.extract(r'(\d+)')[0].fillna(".")

# Merge MTXdosis and Weight from tblPt
merged_data = tbl500.merge(tblpt[["NMID", "MTXdosis", "WT"]], on="NMID", how="left")

# Fill missing MTXdosis with 0
merged_data["MTXdosis"] = merged_data["MTXdosis"].fillna(0)

# Remove rows where VISIT = 0
merged_data = merged_data[merged_data["VISIT"] != 0]

# Create measurement records (CMT = 2)
measurements = merged_data.copy()
measurements["CMT"] = CMT_measurement
measurements["MDV"] = MDV_measurement
measurements["EVID"] = EVID_measurement
measurements["AMT"] = AMT_measurement
measurements["TAD"] = TAD_measurement

# Generate a single dosing record (CMT = 1, 14 days before each observation)
dose_records = []
for _, row in measurements.iterrows():
    dose_time = row["TIME"] - 14  # 14 days before observation
    if dose_time >= 0:  # Ensure valid dose time
        
        
        dose_records.append({
            "NMID": row["NMID"],
            "TIME": dose_time,
            "TAD": TAD_dose,
            "DV": ".",
            "MDV": MDV_dose,
            "CMT": CMT_dose,
            "EVID": EVID_dose,
            "AMT": AMT_dose,
            "SS": SS_dose,
            "II": II_dose,
            "VISIT": ".",
            "TSFD": dose_time - (row["Eerste prik"] - start_date).days,
            "WT": row["WT"],
            "ADA": ".",
            "MTXdosis": row["MTXdosis"]
        })

# Convert dosing records to DataFrame
doses = pd.DataFrame(dose_records)

# Concatenate dose and measurement records
final_dataset = pd.concat([measurements, doses], ignore_index=True)

# Ensure correct column order
column_order = ["NMID", "TIME", "TAD", "DV", "MDV", "CMT", "EVID", "AMT", "SS", "II", "VISIT", "TSFD", "WT", "ADA", "MTXdosis"]
final_dataset = final_dataset[column_order]

# Sort by NMID and TIME
final_dataset = final_dataset.sort_values(by=["NMID", "TIME"]).reset_index(drop=True)

# Replace all empty or missing values in the dataset with "."
final_dataset = final_dataset.applymap(lambda x: "." if pd.isna(x) or x == "" else x)

# Save as CSV
final_dataset.to_csv(os.path.join(OUTPUT_DIR, r'NONMEM_dataset3.csv'), index=False)


## BASE

In [ ]:
# Convert date columns to datetime
start_date = pd.Timestamp("2000-01-01")
tbl500["Afnamedatum"] = pd.to_datetime(tbl500["Afnamedatum"], errors='coerce')
tbl500["Eerste prik"] = pd.to_datetime(tbl500["Eerste prik"], errors='coerce')

# Calculate TIME (days since 01-01-2000) and TSFD (days since Eerste prik)
tbl500["TIME"] = (tbl500["Afnamedatum"] - start_date).dt.days
tbl500["TSFD"] = (tbl500["Afnamedatum"] - tbl500["Eerste prik"]).dt.days

# Extract VISIT (numeric value only)
tbl500["VISIT"] = tbl500["Visit"].str.extract(r'(\d+)').astype(float)

# Assign fixed values
TAD_measurement = 14
TAD_dose = "."
CMT_measurement = 2
CMT_dose = 1
EVID_measurement = 0
EVID_dose = 4
MDV_measurement = 0
MDV_dose = 1
SS_dose = 1
AMT_measurement = "."
AMT_dose = 40
II_dose = 14

# Rename columns for consistency
tbl500 = tbl500.rename(columns={
    "MapNummerOld": "NMID", 
    "adalimumab": "DV", 
    "ARIAtxt": "ADA"
})
tblpt = tblpt.rename(columns={"mapnummer": "NMID", "weight baseline": "WT"})

# Replace missing ADA values with '.' and extract numeric values
tbl500["ADA"] = tbl500["ADA"].astype(str).str.extract(r'(\d+)')[0].fillna("0")

# Replace ADA = "0" with "."
tbl500["ADA"] = tbl500["ADA"].replace("0", ".")


# Merge MTXdosis and Weight from tblPt
merged_data = tbl500.merge(tblpt[["NMID", "MTXdosis", "WT"]], on="NMID", how="left")

# Fill missing MTXdosis with 0
merged_data["MTXdosis"] = merged_data["MTXdosis"].fillna(0)

# Remove rows where VISIT = 0
merged_data = merged_data[merged_data["VISIT"] != 0]

# Create measurement records (CMT = 2)
measurements = merged_data.copy()
measurements["CMT"] = CMT_measurement
measurements["EVID"] = EVID_measurement
measurements["AMT"] = AMT_measurement
measurements["TAD"] = TAD_measurement

measurements["MDV"] = measurements["DV"].apply(lambda x: 1 if pd.isna(x) or x == "." else 0)

# Generate dosing records (CMT = 1, 14 and 28 days before each observation)
dose_records = []
for _, row in measurements.iterrows():
    for offset in [14, 28]:  # Add both 14 and 28 days before measurement
        dose_time = row["TIME"] - offset  # Adjust dose time
        if dose_time >= 0:  # Ensure valid dose time
            
            # Determine SS based on VISIT condition
            SS_value = 1 if row["VISIT"] >= 16 else 0  # 1 for VISIT >= 28, else 0
            II_value = II_dose if SS_value == 1 else "."  # Set II to "." if SS = 0
            EVID_value = EVID_dose if SS_value == 1 else 1

            dose_records.append({
                "NMID": row["NMID"],
                "TIME": dose_time,
                "TAD": TAD_dose,
                "DV": ".",
                "MDV": MDV_dose,
                "CMT": CMT_dose,
                "EVID": EVID_value,
                "AMT": AMT_dose,
                "SS": SS_value,  # Assign SS based on VISIT condition
                "II": II_value,  # Assign II based on SS condition
                "VISIT": ".",
                "TSFD": dose_time - (row["Eerste prik"] - start_date).days,
                "WT": row["WT"],
                "ADA": ".",
                "MTXdosis": row["MTXdosis"]
            })

# Convert dosing records to DataFrame
doses = pd.DataFrame(dose_records)

# Concatenate dose and measurement records
final_dataset = pd.concat([measurements, doses], ignore_index=True)

# Ensure correct column order
column_order = ["NMID", "TIME", "TAD", "DV", "MDV", "CMT", "EVID", "AMT", "SS", "II", "VISIT", "TSFD", "WT", "ADA", "MTXdosis"]
final_dataset = final_dataset[column_order]

# Sort by NMID and TIME
final_dataset = final_dataset.sort_values(by=["NMID", "TIME"]).reset_index(drop=True)

# Replace all empty or missing values in the dataset with "."
final_dataset = final_dataset.applymap(lambda x: "." if pd.isna(x) or x == "" else x)
final_dataset["ADA"] = final_dataset["ADA"].astype(str).replace("nan", ".")

# Save as CSV
final_dataset.to_csv(os.path.join(OUTPUT_DIR, r'NONMEM_dataset4.csv'), index=False)


## Compartment

In [ ]:

# Convert date columns to datetime
start_date = pd.Timestamp("2000-01-01")
tbl500["Afnamedatum"] = pd.to_datetime(tbl500["Afnamedatum"], errors='coerce')
tbl500["Eerste prik"] = pd.to_datetime(tbl500["Eerste prik"], errors='coerce')

# Calculate TIME (days since 01-01-2000) and TSFD (days since Eerste prik)
tbl500["TIME"] = (tbl500["Afnamedatum"] - start_date).dt.days
tbl500["TSFD"] = (tbl500["Afnamedatum"] - tbl500["Eerste prik"]).dt.days

# Extract VISIT (numeric value only)
tbl500["VISIT"] = tbl500["Visit"].str.extract(r'(\d+)').astype(float)

# Assign fixed values
TAD_measurement = 14
TAD_dose = "."
CMT_measurement = 2
CMT_dose = 1
CMT_ada = 4  # New CMT for ADA measurements
EVID_measurement = 0
EVID_dose = 4
MDV_measurement = 0
MDV_dose = 1
SS_dose = 1
AMT_measurement = "."
AMT_dose = 40
II_dose = 14

# Rename columns for consistency
tbl500 = tbl500.rename(columns={
    "MapNummerOld": "NMID", 
    "adalimumab": "DV", 
    "ARIAtxt": "ADA"
})
tblpt = tblpt.rename(columns={"mapnummer": "NMID", "weight baseline": "WT"})

# Extract numeric part from ADA and replace 0 or missing with '.'
tbl500["ADA"] = tbl500["ADA"].astype(str).str.extract(r'(\d+)')[0]
tbl500["ADA"] = tbl500["ADA"].apply(lambda x: "." if pd.isna(x) or x == "0" else x)


# Merge MTXdosis and Weight from tblPt
merged_data = tbl500.merge(tblpt[["NMID", "MTXdosis", "WT"]], on="NMID", how="left")

# Fill missing MTXdosis with 0
merged_data["MTXdosis"] = merged_data["MTXdosis"].fillna(0)

# Remove rows where VISIT = 0
merged_data = merged_data[merged_data["VISIT"] != 0]

# Create measurement records (CMT = 2)
measurements = merged_data.copy()
measurements["CMT"] = CMT_measurement
measurements["EVID"] = EVID_measurement
measurements["AMT"] = AMT_measurement
measurements["TAD"] = TAD_measurement
measurements["MDV"] = measurements["DV"].apply(lambda x: 1 if pd.isna(x) or x == "." else 0)

# Create ADA measurement records (CMT = 4, DV = ADA values)
ada_measurements = measurements.copy()
ada_measurements["DV"] = ada_measurements["ADA"]  # Move ADA values to DV
ada_measurements["CMT"] = CMT_ada

# Generate dosing records (CMT = 1, 14 and 28 days before each observation)
dose_records = []
for _, row in measurements.iterrows():
    for offset in [14, 28]:  # Add both 14 and 28 days before measurement
        dose_time = row["TIME"] - offset  # Adjust dose time
        if dose_time >= 0:  # Ensure valid dose time
            SS_value = 1 if row["VISIT"] >= 16 else 0  # SS = 1 for VISIT >= 28, else 0
            II_value = II_dose if SS_value == 1 else "."
            EVID_value = EVID_dose if SS_value == 1 else 1
            dose_records.append({
                "NMID": row["NMID"],
                "TIME": dose_time,
                "TAD": TAD_dose,
                "DV": ".",
                "MDV": MDV_dose,
                "CMT": CMT_dose,
                "EVID": EVID_value,
                "AMT": AMT_dose,
                "SS": SS_value,
                "II": II_value,
                "VISIT": ".",
                "TSFD": dose_time - (row["Eerste prik"] - start_date).days,
                "WT": row["WT"],
                "ADA": '.',
                "MTXdosis": row["MTXdosis"]
            })

# Convert dosing records to DataFrame
doses = pd.DataFrame(dose_records)

# Concatenate dose, measurement, and ADA records
final_dataset = pd.concat([measurements, ada_measurements, doses], ignore_index=True)

# Ensure correct column order
column_order = ["NMID", "TIME", "TAD", "DV", "MDV", "CMT", "EVID", "AMT", "SS", "II", "VISIT", "TSFD", "WT", "ADA", "MTXdosis"]
final_dataset = final_dataset[column_order]

# Sort by NMID and TIME
final_dataset = final_dataset.sort_values(by=["NMID", "TIME"]).reset_index(drop=True)

# Replace all empty or missing values in the dataset with "."
final_dataset = final_dataset.applymap(lambda x: "." if pd.isna(x) or x == "" else x)
final_dataset["ADA"] = final_dataset["ADA"].astype(str).replace("nan", ".")


# # Update MDV to 1 where CMT == 4
# final_dataset.loc[final_dataset["CMT"] == 4, "MDV"] = 1

final_dataset["MDV"] = final_dataset["DV"].apply(lambda x: 1 if x == "." else 0)

# # Save the modified dataset
final_dataset.to_csv(os.path.join(OUTPUT_DIR, r'NONMEM_dataset6.csv'), index=False)


### LOGscale ADA

In [ ]:


# Load your dataset (adjust the path if needed)
df = final_dataset

# Backup original ADA column
ada_original = df['ADA'].copy()

# Convert ADA to numeric (".", etc. become NaN)
df['ADA'] = pd.to_numeric(df['ADA'], errors='coerce')

# Apply log transformation using log1p (safe for zeros)
df['ADA'] = df['ADA'].apply(lambda x: np.log1p(x) if pd.notnull(x) and x >= 0 else x)

# Restore "." where it was originally
df.loc[ada_original == ".", 'ADA'] = "."



# Save or preview the updated dataset
df.to_csv(os.path.join(OUTPUT_DIR, r'fnm_adl_3.csv'), index=False)



## CASE 2

In [ ]:
import pandas as pd
from datetime import timedelta

# Load your data (assumed already loaded in tbl500 and tblpt)

# Filter observations with positive days
tbl500 = tbl500[tbl500["Days"] > 0]

# Ensure required columns exist
if 'MapNummerOld' not in tbl500.columns:
    raise KeyError("Column 'MapNummerOld' not found in tbl500.xlsx")
if 'mapnummer' not in tblpt.columns:
    raise KeyError("Column 'mapnummer' not found in tblPt.xlsx")

# Convert date columns to datetime
start_date = pd.Timestamp("2000-01-01")
tbl500["Afnamedatum"] = pd.to_datetime(tbl500["Afnamedatum"], errors='coerce')
tbl500["Eerste prik"] = pd.to_datetime(tbl500["Eerste prik"], errors='coerce')

# Calculate TIME and TSFD
tbl500["TIME"] = (tbl500["Afnamedatum"] - start_date).dt.days
tbl500["TSFD"] = (tbl500["Afnamedatum"] - tbl500["Eerste prik"]).dt.days

# Extract VISIT number
tbl500["VISIT"] = tbl500["Visit"].str.extract(r'(\d+)').astype(float)

# Rename columns for consistency
tbl500 = tbl500.rename(columns={
    "MapNummerOld": "NMID", 
    "adalimumab": "DV", 
    "ARIAtxt": "ADA"
})
tblpt = tblpt.rename(columns={"mapnummer": "NMID", "weight baseline": "WT"})

# Handle ADA values
tbl500["ADA"] = tbl500["ADA"].astype(str).str.extract(r'(\d+)')[0].fillna(".")

# Merge MTXdosis and WT
merged_data = tbl500.merge(tblpt[["NMID", "MTXdosis", "WT"]], on="NMID", how="left")
merged_data["MTXdosis"] = merged_data["MTXdosis"].fillna(0)

# Create observation records
measurements = merged_data.copy()
measurements["CMT"] = 2
measurements["MDV"] = 0
measurements["EVID"] = 0
measurements["AMT"] = "."
measurements["TAD"] = 14
measurements["SS"] = 0
measurements["II"] = 0

# Create scheduled dose records
scheduled_doses = []
dose_interval = 14  # every 14 days

for nmid, group in merged_data.groupby("NMID"):
    eerste_prik = group["Eerste prik"].iloc[0]
    last_obs_date = group["Afnamedatum"].max()
    dose_date = eerste_prik

    while dose_date <= last_obs_date:
        scheduled_doses.append({
            "NMID": nmid,
            "DOSE_DATE": dose_date
        })
        dose_date += timedelta(days=dose_interval)

scheduled_doses_df = pd.DataFrame(scheduled_doses)
scheduled_doses_df["TIME"] = (scheduled_doses_df["DOSE_DATE"] - start_date).dt.days
scheduled_doses_df["USED"] = False

# Move the closest dose to each observation time
obs_data = merged_data.copy()
obs_data["OBS_TIME"] = obs_data["TIME"]

for _, obs in obs_data.iterrows():
    nmid = obs["NMID"]
    obs_time = obs["OBS_TIME"]

    doses_for_nmid = scheduled_doses_df[
        (scheduled_doses_df["NMID"] == nmid) &
        (~scheduled_doses_df["USED"])
    ]

    before_obs = doses_for_nmid[doses_for_nmid["TIME"] <= obs_time]
    if before_obs.empty:
        continue

    closest_dose_idx = (obs_time - before_obs["TIME"]).idxmin()
    scheduled_doses_df.loc[closest_dose_idx, "TIME"] = obs_time
    scheduled_doses_df.loc[closest_dose_idx, "USED"] = True

# Finalize dose records
final_doses = scheduled_doses_df.copy()
final_doses = final_doses.merge(tblpt[["NMID", "MTXdosis", "WT"]], on="NMID", how="left")
final_doses["TSFD"] = "."
final_doses["DV"] = "."
final_doses["MDV"] = 1
final_doses["CMT"] = 1
final_doses["EVID"] = 1
final_doses["AMT"] = 40
final_doses["SS"] = 0
final_doses["II"] = 0
final_doses["VISIT"] = "."
final_doses["ADA"] = "."
final_doses["TAD"] = "."

# Select correct column order
final_doses = final_doses[[
    "NMID", "TIME", "TAD", "DV", "MDV", "CMT", "EVID", "AMT", "SS", "II",
    "VISIT", "TSFD", "WT", "ADA", "MTXdosis"
]]

# Same for observations
measurements = measurements[[
    "NMID", "TIME", "TAD", "DV", "MDV", "CMT", "EVID", "AMT", "SS", "II",
    "VISIT", "TSFD", "WT", "ADA", "MTXdosis"
]]

# Combine both
final_dataset = pd.concat([measurements, final_doses], ignore_index=True)
final_dataset = final_dataset.sort_values(by=["NMID", "TIME"]).reset_index(drop=True)

# Replace missing or blank values with "."
final_dataset = final_dataset.map(lambda x: "." if pd.isna(x) or x == "" else x)

final_dataset["SS"] = "."
final_dataset["II"] = "."

final_dataset["MDV"] = final_dataset["DV"].apply(lambda x: 1 if x == "." else 0)


# Save to CSV
final_dataset.to_csv(os.path.join(OUTPUT_DIR, r'NONMEM_dataset2.csv'), index=False)


## First fix

In [ ]:
import pandas as pd
from datetime import timedelta

# Ensure required columns exist
if 'MapNummerOld' not in tbl500.columns:
    raise KeyError("Column 'MapNummerOld' not found in tbl500.xlsx")
if 'mapnummer' not in tblpt.columns:
    raise KeyError("Column 'mapnummer' not found in tblPt.xlsx")

# Convert dates
start_date = pd.Timestamp("2000-01-01")
tbl500["Afnamedatum"] = pd.to_datetime(tbl500["Afnamedatum"], errors='coerce')
tbl500["Eerste prik"] = pd.to_datetime(tbl500["Eerste prik"], errors='coerce')

# Calculate TIME and TSFD
tbl500["TIME"] = (tbl500["Afnamedatum"] - start_date).dt.days
tbl500["TSFD"] = (tbl500["Afnamedatum"] - tbl500["Eerste prik"]).dt.days

# Extract VISIT
tbl500["VISIT"] = tbl500["Visit"].str.extract(r'(\d+)').astype(float)

# Rename for consistency
tbl500 = tbl500.rename(columns={
    "MapNummerOld": "NMID", 
    "adalimumab": "DV", 
    "ARIAtxt": "ADA"
})
tblpt = tblpt.rename(columns={"mapnummer": "NMID", "weight baseline": "WT"})

# Handle ADA
tbl500["ADA"] = tbl500["ADA"].astype(str).str.extract(r'(\d+)')[0].fillna(".")

# Merge
merged_data = tbl500.merge(tblpt[["NMID", "MTXdosis", "WT"]], on="NMID", how="left")
merged_data["MTXdosis"] = merged_data["MTXdosis"].fillna(0)

# Observation records
measurements = merged_data.copy()
measurements["CMT"] = 2
measurements["MDV"] = 0
measurements["EVID"] = 0
measurements["AMT"] = "."
measurements["TAD"] = "."
measurements["SS"] = 0
measurements["II"] = 0

# Generate scheduled doses
scheduled_doses = []
dose_interval = 14

for nmid, group in merged_data.groupby("NMID"):
    eerste_prik = group["Eerste prik"].iloc[0]
    last_obs_date = group["Afnamedatum"].max()
    dose_date = eerste_prik

    while dose_date <= last_obs_date + timedelta(days=28):
        scheduled_doses.append({
            "NMID": nmid,
            "DOSE_DATE": dose_date
        })
        dose_date += timedelta(days=dose_interval)

scheduled_doses_df = pd.DataFrame(scheduled_doses)
scheduled_doses_df["TIME"] = (scheduled_doses_df["DOSE_DATE"] - start_date).dt.days
scheduled_doses_df["USED"] = False

# Move best dose to obs time for TAD ≈ 14
moved_doses = []
obs_data = merged_data.copy()
obs_data["OBS_TIME"] = obs_data["TIME"]

for _, obs in obs_data.iterrows():
    nmid = obs["NMID"]
    obs_time = obs["OBS_TIME"]

    # Available doses for this NMID
    all_doses = scheduled_doses_df[
        (scheduled_doses_df["NMID"] == nmid) &
        (~scheduled_doses_df["USED"])
    ].sort_values("TIME")

    lower = all_doses[all_doses["TIME"] <= obs_time]
    upper = all_doses[all_doses["TIME"] > obs_time]

    def simulate_tad(move_idx):
        move_time = all_doses.loc[move_idx, "TIME"]
        # Simulate moving this dose to obs time
        temp_doses = all_doses.drop(index=move_idx).copy()
        previous_doses = temp_doses[temp_doses["TIME"] < obs_time]
        if previous_doses.empty:
            return float("inf")
        last_real_dose_time = previous_doses["TIME"].max()
        return abs((obs_time - last_real_dose_time) - 14)

    best_idx = None
    best_tad_diff = float("inf")

    if not lower.empty:
        idx = (obs_time - lower["TIME"]).idxmin()
        tad_diff = simulate_tad(idx)
        if tad_diff < best_tad_diff:
            best_idx = idx
            best_tad_diff = tad_diff

    if not upper.empty:
        idx = (upper["TIME"] - obs_time).idxmin()
        tad_diff = simulate_tad(idx)
        if tad_diff < best_tad_diff:
            best_idx = idx
            best_tad_diff = tad_diff

    if best_idx is not None:
        scheduled_doses_df.loc[best_idx, "TIME"] = obs_time
        scheduled_doses_df.loc[best_idx, "USED"] = True

# Finalize dose records
final_doses = scheduled_doses_df.copy()
final_doses = final_doses.merge(tblpt[["NMID", "MTXdosis", "WT"]], on="NMID", how="left")
final_doses["TSFD"] = "."
final_doses["DV"] = "."
final_doses["MDV"] = 1
final_doses["CMT"] = 1
final_doses["EVID"] = 1
final_doses["AMT"] = 40
final_doses["SS"] = 0
final_doses["II"] = 0
final_doses["VISIT"] = "."
final_doses["ADA"] = "."
final_doses["TAD"] = "."

# Column order
final_doses = final_doses[[
    "NMID", "TIME", "TAD", "DV", "MDV", "CMT", "EVID", "AMT", "SS", "II",
    "VISIT", "TSFD", "WT", "ADA", "MTXdosis"
]]
measurements = measurements[[
    "NMID", "TIME", "TAD", "DV", "MDV", "CMT", "EVID", "AMT", "SS", "II",
    "VISIT", "TSFD", "WT", "ADA", "MTXdosis"
]]

# Combine & sort
combined = pd.concat([measurements, final_doses], ignore_index=True)
combined = combined.sort_values(by=["NMID", "TIME"]).reset_index(drop=True)

# TAD calculation based on real previous dose
combined["TAD"] = "."
for nmid, group in combined.groupby("NMID"):
    group = group.sort_values("TIME")
    last_dose_time = None
    tad_list = []

    for _, row in group.iterrows():
        if row["EVID"] == 1:
            last_dose_time = row["TIME"]
            tad_list.append(".")
        elif row["EVID"] == 0:
            if last_dose_time is not None and int(row["TIME"]) > int(last_dose_time):
                tad_val = int(row["TIME"]) - int(last_dose_time)
                tad_list.append(tad_val)
            else:
                tad_list.append(".")
        else:
            tad_list.append(".")

    combined.loc[group.index, "TAD"] = tad_list

# Final formatting
final_dataset = combined.map(lambda x: "." if pd.isna(x) or x == "" else x)
final_dataset["SS"] = "."
final_dataset["II"] = "."
final_dataset["MDV"] = final_dataset["DV"].apply(lambda x: 1 if x == "." else 0)

# Save
final_dataset.to_csv(os.path.join(OUTPUT_DIR, r'NONMEM_dataset2.csv'), index=False)


## BASED

In [ ]:
# Create a new DataFrame to store the visit-based timing
visit_based_records = []

# Only keep rows with valid VISIT info
valid_rows = combined[combined["VISIT"] != "."].copy()
valid_rows["VISIT"] = valid_rows["VISIT"].astype(float)

# Define fixed time per visit
visit_day_interval = 7

# Build standardized schedule
for nmid, group in valid_rows.groupby("NMID"):
    for _, row in group.iterrows():
        visit = int(row["VISIT"])
        time = visit * visit_day_interval
        new_row = row.copy()
        new_row["TIME"] = time
        visit_based_records.append(new_row)

# Add dose events at every 14 days up to max TIME per subject
standard_doses = []
dose_interval = 14

for nmid, group in valid_rows.groupby("NMID"):
    max_visit = group["VISIT"].max()
    max_time = int(max_visit * visit_day_interval)
    dose_times = list(range(0, max_time + 1, dose_interval))

    for time in dose_times:
        dose_event = {
            "NMID": nmid,
            "TIME": time,
            "TAD": ".",
            "DV": ".",
            "MDV": 1,
            "CMT": 1,
            "EVID": 1,
            "AMT": 40,
            "SS": ".",
            "II": ".",
            "VISIT": ".",
            "TSFD": ".",
            "WT": group["WT"].iloc[0],
            "ADA": ".",
            "MTXdosis": group["MTXdosis"].iloc[0]
        }
        standard_doses.append(dose_event)

# Combine and sort
visit_based_df = pd.DataFrame([r.to_dict() for r in visit_based_records] + standard_doses)
visit_based_df = visit_based_df.sort_values(by=["NMID", "TIME"]).reset_index(drop=True)

# Final format tweaks
visit_based_df = visit_based_df.map(lambda x: "." if pd.isna(x) or x == "" else x)
visit_based_df["MDV"] = visit_based_df["DV"].apply(lambda x: 1 if x == "." else 0)


# Remove measurements at TIME 0 (EVID 0), keep dose at TIME 0 (EVID 1)
visit_based_df = visit_based_df[~((visit_based_df["TIME"] == 0) & (visit_based_df["EVID"] == 0))]

# Save
visit_based_df.to_csv(os.path.join(OUTPUT_DIR, r'NONMEM_dataset_visit_based.csv'), index=False)


In [ ]:
# Filter for original observation rows (EVID 0, MDV 0)
obs_rows = visit_based_df[(visit_based_df["EVID"] == 0) & (visit_based_df["MDV"] == 0)].copy()

# Create new ADA rows based on those
ada_rows = obs_rows.copy()

# Replace DV with ADA
ada_rows["DV"] = ada_rows["ADA"]

# Optional: Change CMT to indicate ADA (use 3 if that fits your model setup)
ada_rows["CMT"] = 4

# Make sure MDV is 0 for ADA rows with valid values, 1 for "."
ada_rows["MDV"] = ada_rows["DV"].apply(lambda x: 1 if x == "." else 0)

# Combine with original dataset
visit_based_with_ada = pd.concat([visit_based_df, ada_rows], ignore_index=True)

# Sort by NMID and TIME again
visit_based_with_ada = visit_based_with_ada.sort_values(by=["NMID", "TIME"]).reset_index(drop=True)

# Save to new file
visit_based_with_ada.to_csv(os.path.join(OUTPUT_DIR, r'NONMEM_dataset_visit_based_with_ADA.csv'), index=False)
